In [6]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd
import json
from datetime import datetime
import time
import re
from bs4 import BeautifulSoup

In [2]:
options = Options()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument('--user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36')
driver = webdriver.Chrome(options=options)
driver.implicitly_wait(2)

In [7]:
def parse_with_soup(html):
    try:
        soup = BeautifulSoup(html, 'html.parser')
        return soup
    except Exception as e:
        print(f"파싱 실패: {e}")
        return None

In [ ]:
driver.get("https://kmong.com/category/661")

WebDriverWait(driver, 10).until(EC.visibility_of_all_elements_located((By.CLASS_NAME, 'css-enj2mu'))) # 서비스 수 찾기
flag_found_service = driver.find_element(By.CLASS_NAME, "css-enj2mu").text
if flag_found_service == 0:
    pass
else:
    service_amount = flag_found_service

    WebDriverWait(driver, 10).until(EC.visibility_of_all_elements_located((By.CLASS_NAME, 'edqw2x10'))) # 서비스 다 보일때까지 대기

    html = driver.page_source
    soup = parse_with_soup(html)
    